# Santender Bank

## About Santander
Banco Santander, S.A., doing business as Santander Group, is a Spanish multinational commercial bank and financial services company based in Madrid and Santander in Spain. Additionally, Santander maintains a presence in all global financial centres as the 16th-largest banking institution in the world.

## About the challenge
Ready to make a downpayment on your first house? Or looking to leverage the equity in the home you have? To support needs for a range of financial decisions, Santander Bank offers a lending hand to their customers through personalized product recommendations.

Under their current system, a small number of Santander’s customers receive many recommendations while many others rarely see any resulting in an uneven customer experience. In their second competition, Santander is challenging Kagglers to predict which products their existing customers will use in the next month based on their past behavior and that of similar customers.

With a more effective recommendation system in place, Santander can better meet the individual needs of all customers and ensure their satisfaction no matter where they are in life.

In the competition, you are provided with 1.5 years of customers behavior data from Santander bank to predict what new products customers will purchase. The data starts at 2015-01-28 and has monthly records of products a customer has, such as “credit card”, “savings account”, etc. You will predict what additional products a customer will get in the last month, 2016-06-28, in addition to what they already have at 2016-05-28. These products are the columns named: ind_(xyz)_ult1, which are the columns #25 - #48 in the training data. You will predict what a customer will buy in addition to what they already had at 2016-05-28.

In [ ]:
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())

In [1]:
#The libraries used for data reading ,plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
plt.style.use("fivethirtyeight")
#libraries used for Association rule mining
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
#Reading the data into python.
train=pd.read_csv("train_ver2.csv",error_bad_lines=False,na_values=[" "])

C:\Users\admin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
test=pd.read_csv("test_ver2.csv",low_memory=False,na_values=[""," ","?"])

In [8]:
#viewing the data
train.shape

(13647309, 48)

In [9]:
train.describe()

,ncodpers,ind_nuevo,indrel,tipodom,cod_prov,ind_actividad_cliente,renta,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
count,1.364731e+07,1.361958e+07,1.361958e+07,13619574.0,1.355372e+07,1.361958e+07,1.085293e+07,1.364731e+07,1.364731e+07,1.364731e+07,...,1.364731e+07,1.364731e+07,1.364731e+07,1.364731e+07,1.364731e+07,1.364731e+07,1.364731e+07,1.363125e+07,1.363125e+07,1.364731e+07
mean,8.349042e+05,5.956184e-02,1.178399e+00,1.0,2.657147e+01,4.578105e-01,1.342543e+05,1.022912e-04,2.315475e-05,6.554837e-01,...,5.886582e-03,9.170965e-03,2.627404e-03,5.253636e-02,4.438868e-02,2.560761e-02,3.847718e-03,5.472434e-02,5.942854e-02,1.279162e-01
std,4.315650e+05,2.366733e-01,4.177469e+00,0.0,1.278402e+01,4.982169e-01,2.306202e+05,1.011340e-02,4.811883e-03,4.752103e-01,...,7.649791e-02,9.532502e-02,5.119083e-02,2.231060e-01,2.059571e-01,1.579616e-01,6.191053e-02,2.274414e-01,2.364250e-01,3.339965e-01
min,1.588900e+04,0.000000e+00,1.000000e+00,1.0,1.000000e+00,0.000000e+00,1.202730e+03,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.528130e+05,0.000000e+00,1.000000e+00,1.0,1.500000e+01,0.000000e+00,6.871098e+04,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,9.318930e+05,0.000000e+00,1.000000e+00,1.0,2.800000e+01,0.000000e+00,1.018500e+05,0.000000e+00,0.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.199286e+06,0.000000e+00,1.000000e+00,1.0,3.500000e+01,1.000000e+00,1.559560e+05,0.000000e+00,0.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.553689e+06,1.000000e+00,9.900000e+01,1.0,5.200000e+01,1.000000e+00,2.889440e+07,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [6]:
#calculating the parcentage of missing values in the each column.
(train.isna().sum()/len(train))*100

fecha_dato                0.000000
ncodpers                  0.000000
ind_empleado              0.203220
pais_residencia           0.203220
sexo                      0.203732
age                       0.000000
fecha_alta                0.203220
ind_nuevo                 0.203220
antiguedad                0.000000
indrel                    0.203220
ult_fec_cli_1t           99.818330
indrel_1mes               1.097513
tiprel_1mes               1.097513
indresi                   0.203220
indext                    0.203220
conyuemp                 99.986752
canal_entrada             1.363829
indfall                   0.203220
tipodom                   0.203227
cod_prov                  0.685784
nomprov                   0.685784
ind_actividad_cliente     0.203220
renta                    20.475648
segmento                  1.387585
ind_ahor_fin_ult1         0.000000
ind_aval_fin_ult1         0.000000
ind_cco_fin_ult1          0.000000
ind_cder_fin_ult1         0.000000
ind_cno_fin_ult1    

In these two columns 99% of the data is missing so we are removing these two columns .ult_fec_cli_1t("Last date as primary customer (if he isn't at the end of the month)") ,canal_entrada("Spouse index. 1 if the customer is spouse of an employee")

In [10]:
#Removing the unwanted columns.
train=train.drop(['ult_fec_cli_1t', 'conyuemp'], axis = 1)

In [8]:
#renta :- Gross income of the household
#train.info(memory_usage='deep')
#train[train['renta'].isna()==True]

checking why NA 's there in this column 
renta :- Gross income of the household

In [9]:
pd.set_option('display.max_columns', 500)
train[train['renta'].isna()==True]

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,indrel_1mes,tiprel_1mes,indresi,indext,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,1,A,S,N,KHE,N,1.0,50.0,ZARAGOZA,1.0,NaN,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
8,2015-01-28,1050619,N,ES,H,24,2012-08-10,0.0,35,1.0,1,I,S,N,KHE,N,1.0,20.0,GIPUZKOA,0.0,NaN,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
11,2015-01-28,1050622,N,ES,H,23,2012-08-10,0.0,35,1.0,1,I,S,N,KHE,N,1.0,17.0,GIRONA,0.0,NaN,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
14,2015-01-28,1050625,N,ES,V,23,2012-08-10,0.0,35,1.0,1,A,S,N,KHE,N,1.0,49.0,ZAMORA,1.0,NaN,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
15,2015-01-28,1050626,N,ES,V,23,2012-08-10,0.0,35,1.0,1,A,S,N,KHE,N,1.0,8.0,BARCELONA,1.0,NaN,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
19,2015-01-28,1050605,N,ES,V,23,2012-08-10,0.0,35,1.0,1,I,S,N,KHE,N,1.0,45.0,TOLEDO,0.0,NaN,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
22,2015-01-28,1050588,N,ES,H,22,2012-08-10,0.0,35,1.0,1,I,S,N,KHE,N,1.0,9.0,BURGOS,1.0,NaN,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
25,2015-01-28,1050592,N,ES,H,22,2012-08-10,0.0,35,1.0,1,I,S,N,KHD,N,1.0,31.0,NAVARRA,0.0,NaN,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
28,2015-01-28,1050597,N,ES,H,23,2012-08-10,0.0,35,1.0,1,A,S,N,KHE,N,1.0,45.0,TOLEDO,0.0,NaN,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
37,2015-01-28,1050628,N,ES,V,22,2012-08-10,0.0,35,1.0,1,I,S,N,KHE,N,1.0,25.0,LERIDA,0.0,NaN,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0


In [10]:
a=train.head(5)
a
a["segmento"].str.split("-", n = 1, expand = True) 

,0,1
0,02,PARTICULARES
1,03,UNIVERSITARIO
2,03,UNIVERSITARIO
3,03,UNIVERSITARIO
4,03,UNIVERSITARIO


In [11]:
#a=train.head(5)
#a
#from googletrans import Translator
#translator = Translator()
#train['eng'] = train['segmento'].apply(translator.translate, src='id' ,dest='en')

In [12]:
#pd.set_option('display.max_columns', 500)
a[['eng','segmento']]

KeyError: "['eng'] not in index"

In [13]:
train['fecha_dato'].max()

'2016-05-28'

In [14]:
len(train['ncodpers'].unique())

956645

In [15]:
#train.columns
#from googletrans import Translator
#translator = Translator()
#list(map(translator,train.columns))

In [11]:
#Subsetting the data base on the date.
train['fecha_dato']=pd.to_datetime(train['fecha_dato'], format="%Y-%m-%d")
train_sample=train[train['fecha_dato'] >= pd.to_datetime("2015-05-01", format="%Y-%m-%d")]

In [17]:
# % of users we got after subsetting the original data. 
(len(train_sample['ncodpers'].unique())/len(train['ncodpers'].unique()))*100

99.62838879626193

In [18]:
(train_sample.isna().sum()/len(train_sample))*100

fecha_dato                0.000000
ncodpers                  0.000000
ind_empleado              0.049017
pais_residencia           0.049017
sexo                      0.049556
age                       0.000000
fecha_alta                0.049017
ind_nuevo                 0.049017
antiguedad                0.000000
indrel                    0.049017
indrel_1mes               1.145095
tiprel_1mes               1.145095
indresi                   0.049017
indext                    0.049017
canal_entrada             1.470002
indfall                   0.049017
tipodom                   0.049026
cod_prov                  0.508357
nomprov                   0.508357
ind_actividad_cliente     0.049017
renta                    21.111198
segmento                  1.498669
ind_ahor_fin_ult1         0.000000
ind_aval_fin_ult1         0.000000
ind_cco_fin_ult1          0.000000
ind_cder_fin_ult1         0.000000
ind_cno_fin_ult1          0.000000
ind_ctju_fin_ult1         0.000000
ind_ctma_fin_ult1   

In [12]:
#train_sample[train_sample['ind_nomina_ult1'].isna()==True].head(4)
train_sample_na=train_sample[train_sample['ind_nomina_ult1'].isna()==False]

There are some logic to imputing the NA 's value ,but for this project i am just removing all the na values rows

In [20]:
train_sample.shape

(11134882, 46)

In [21]:
train_sample.isna().sum()

fecha_dato                     0
ncodpers                       0
ind_empleado                5458
pais_residencia             5458
sexo                        5518
age                            0
fecha_alta                  5458
ind_nuevo                   5458
antiguedad                     0
indrel                      5458
indrel_1mes               127505
tiprel_1mes               127505
indresi                     5458
indext                      5458
canal_entrada             163683
indfall                     5458
tipodom                     5459
cod_prov                   56605
nomprov                    56605
ind_actividad_cliente       5458
renta                    2350707
segmento                  166875
ind_ahor_fin_ult1              0
ind_aval_fin_ult1              0
ind_cco_fin_ult1               0
ind_cder_fin_ult1              0
ind_cno_fin_ult1               0
ind_ctju_fin_ult1              0
ind_ctma_fin_ult1              0
ind_ctop_fin_ult1              0
ind_ctpp_f

# Association rule mining.

we are doing association rule mining on all the products.

In [22]:
#Creating data to find association between the products.
association_data=train_sample_na[["ncodpers",'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']]

#converting the user id into string
#association_data['ncodpers']=association_data['ncodpers'].astype('str')

In [23]:
#Now setting customer Id as index 
association_data['ncodpers']=association_data['ncodpers'].astype('str')
#association_data.head(2)

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
#setting the customer id as index
association_data=association_data.set_index('ncodpers')

In [25]:
#converting float data to (int)
association_data['ind_nomina_ult1']=association_data['ind_nomina_ult1'].astype('int')
association_data['ind_nom_pens_ult1']=association_data['ind_nom_pens_ult1'].astype('int')

In [26]:
association_data.head(5)

,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
ncodpers,,,,,,,,,,,,,,,,,,,,,,,,
1061260,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1061283,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1061284,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1061336,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1061286,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
frequent_itemsets = apriori(association_data, min_support=0.07, use_colnames=True)
#rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
frequent_itemsets.sort_values(['support'], ascending=[False])

,support,itemsets
0,0.628101,(ind_cco_fin_ult1)
4,0.122192,(ind_recibo_ult1)
2,0.120145,(ind_ctop_fin_ult1)
3,0.079343,(ind_ecue_fin_ult1)
5,0.079243,"(ind_ctop_fin_ult1, ind_cco_fin_ult1)"
1,0.077415,(ind_cno_fin_ult1)


In [28]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ind_ctop_fin_ult1),(ind_cco_fin_ult1),0.120145,0.628101,0.079243,0.659559,1.050084,0.00378,1.092403


In [29]:
lift=association_rules(frequent_itemsets, metric="lift", min_threshold=1)
lift

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ind_ctop_fin_ult1),(ind_cco_fin_ult1),0.120145,0.628101,0.079243,0.659559,1.050084,0.00378,1.092403
1,(ind_cco_fin_ult1),(ind_ctop_fin_ult1),0.628101,0.120145,0.079243,0.126163,1.050084,0.00378,1.006886


# EDA

In [30]:
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
plt.style.use("fivethirtyeight")

In [13]:
train_sample.head(1)

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
2512427,2015-05-28,1061260,N,ES,H,24,2012-09-17,0.0,34,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [14]:
train_sample['fecha_dato'].min()

Timestamp('2015-05-28 00:00:00')

In [27]:
train_sample['segmento']=train_sample["segmento"].str.split("-", n = 1, expand = True).iloc[:,1]

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [29]:
pd.set_option('display.max_columns', 500)
train_sample.head(2)

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,indrel_1mes,tiprel_1mes,indresi,indext,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
2512427,2015-05-28,1061260,N,ES,H,24,2012-09-17,0.0,34,1.0,1,I,S,N,KHE,N,1.0,28.0,MADRID,0.0,396426.42,UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
2512428,2015-05-28,1061283,N,ES,H,22,2012-09-17,0.0,34,1.0,1,I,S,N,KHE,N,1.0,28.0,MADRID,0.0,155917.08,UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0


In [ ]:
import pandas as pd
from googletrans import Translator

translator = Translator()
train_sample["segmento"].map(lambda x: translator.translate(x, src="zh-TW", dest="en").text)